### 사업보고서 "재무제표 주석" 문서 URL

In [1]:
import pandas as pd

# 사업보고서 목록
# file_path = "D:/PythonProject/data-gatherer/dart_fs_notes/dart_rpt/A_rpt_all.csv"

# 사업보고서 목록 중 제조업
file_path = "D:/PythonProject/data-gatherer/dart_fs_notes/dart_rpt/A_rpt_all_mfg.csv"

df_rpt_all = pd.read_csv(file_path, dtype=object)

* 대상 목록 확인

In [2]:
df_rpt_all[df_rpt_all['corp_name'].str.contains("리츠")]

,corp_code,corp_name,stock_code,corp_cls,report_nm,rcept_no,flr_nm,rcept_dt,rm,induty_code


In [3]:
df_rpt_all['corp_cls'].value_counts()

corp_cls
K    2674
Y    1801
E    1024
N     210
Name: count, dtype: int64

In [4]:
# 회사코드 정렬
df_rpt_all = df_rpt_all.sort_values(by=['corp_code'], ascending=True)
# 유가증권 기업 + 결산년도 2017~2021
list_all_codes = df_rpt_all.loc[(df_rpt_all.corp_cls == 'Y') & (df_rpt_all.report_nm.str[-8:-4] >= '2017') & (df_rpt_all.report_nm.str[-8:-4] <= '2021'), ]['corp_code'].unique()
len(list_all_codes)

274

* 이미 재무제표 주석 문서 URL을 찾은 것은 대상에서 제외

In [5]:
import os.path
# 이미 재무제표 주석 문서 URL을 찾은 회사코드 목록
file_path = "D:/PythonProject/data-gatherer/dart_fs_notes/fs_notes_urls/fs_notes_url_all.csv"
if os.path.exists(file_path):
    fs_notes_all = pd.read_csv(file_path, dtype=object)
    list_own_codes = fs_notes_all['corp_code'].unique()
else:
    list_own_codes = []
    
len(list_own_codes)

342

In [14]:
# 리스트에서 중복 제거 (이미 corp info가 있는 경우 대상에서 제외)
list_target_codes = list(set(list_all_codes) - set(list_own_codes))
len(list_target_codes)

0

* 리스트 분할

In [15]:
list_cnt = len(list_target_codes)
n = 100
list_of_lists = [list_target_codes[i * n:(i + 1) * n] for i in range((list_cnt + n - 1) // n )] 
len(list_of_lists)

0

* 재무제표 주석 문서 가져오기

In [8]:
import traceback
import datetime
from time import sleep
import OpenDartReader
api_key = 'f2e08d4ed3de0ba3d5cbf59c04c223e02b1751a2'
dart = OpenDartReader(api_key)

import sys
module_path = "D:\PythonProject\data-gatherer\dart_fs_notes"
sys.path.append(module_path)
import myutil

# 파일 저장 위치
path_dir = "D:/PythonProject/data-gatherer/dart_fs_notes/fs_notes_urls/"

In [9]:
for corp_codes in list_of_lists:
    for corp_code in corp_codes:
        df_biz_rpt = df_rpt_all.loc[df_rpt_all.corp_code == corp_code,].sort_values(by=['rcept_dt'], ascending=True)
        rpt_count = df_biz_rpt.shape[0]
        subreports = []
        for i, row in df_biz_rpt.iterrows():
            sleep(1.0)
            try:
                rpt_name = row['report_nm']
                rpt_num = row['rcept_no']        
                rpt_url = 'http://dart.fss.or.kr/dsaf001/main.do?rcpNo=' + rpt_num
                df_doc = dart.sub_docs(rpt_num)     
                df_notes = df_doc.loc[(df_doc.title.str.contains('주석')) & (~df_doc.title.str.contains('연결')),]
                if df_notes.size == 0:
                    subreports.append({'corp_code' : row['corp_code'], 'stock_code' : row['stock_code'], 'corp_cls' : row['corp_cls'], 'corp_name' : row['corp_name'],
                                       'rpt_num' : rpt_num, 'rpt_name' : rpt_name, 'rpt_url' : rpt_url,
                                       'doc_title' : None, 'doc_url' : None, 'doc_result' : 'No Financial Statement Notes'})                 
                else:
                    rpt_title = df_notes.iloc[0]['title']
                    rpt_notes_url = df_notes.iloc[0]['url']
                    subreports.append({'corp_code' : row['corp_code'], 'stock_code' : row['stock_code'], 'corp_cls' : row['corp_cls'], 'corp_name' : row['corp_name'],
                                       'rpt_num' : rpt_num, 'rpt_name' : rpt_name, 'rpt_url' : rpt_url, 
                                       'doc_title' : rpt_title, 'doc_url' : rpt_notes_url, 'doc_result' : 'OK'}) 
            except Exception as e:
                print(row['rcept_no'])
                print(row['report_nm'])
                print('http://dart.fss.or.kr/dsaf001/main.do?rcpNo=' + row['rcept_no'])
                err_msg = traceback.format_exc()
                print(err_msg)
                break
        subrpt_count = len(subreports)        
        if subrpt_count == 0:
            break
        elif subrpt_count == rpt_count:
            # 해당 기업의 사업보고서 URL 목록을 파일로 저장
            df_subreports = pd.DataFrame(subreports)
            file_name = corp_code + "_url_" + myutil.now_dt_str() + '.csv'
            df_subreports.to_csv(path_dir + file_name, index=False)
        else:
            continue